In [2]:
import pandas as pd 
import numpy as np

In [3]:
df = pd.read_csv('datasets/train.csv', index_col='PassengerId')
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [6]:
df.isna().mean()*100

Survived     0.000000
Pclass       0.000000
Name         0.000000
Sex          0.000000
Age         19.865320
SibSp        0.000000
Parch        0.000000
Ticket       0.000000
Fare         0.000000
Cabin       77.104377
Embarked     0.224467
dtype: float64

In [7]:
# Remove passengersId, Name, Ticket and Cabin
titanic = df.reset_index()
removed_col = ['PassengerId', 'Name', 'Cabin', 'Ticket']
titanic = titanic.drop(removed_col, axis=1)

# fill Age with median
titanic['Age'] = titanic['Age'].fillna(titanic['Age'].median())

# fill Embarked with mode
embarked_mode = titanic['Embarked'].mode()[0]
titanic['Embarked'] = titanic['Embarked'].fillna(embarked_mode)
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [131]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

X = titanic.drop(['Survived'], axis=1)
y = titanic['Survived']

# Label econde sex col
le = LabelEncoder()
X['Sex'] = le.fit_transform(X['Sex'])


# Onehot encode Embarked col
ohe = OneHotEncoder(drop='first', sparse_output=False)
embarked_encoded = ohe.fit_transform(X[['Embarked']])
embarked_df = pd.DataFrame(embarked_encoded, columns=ohe.get_feature_names_out(['Embarked']))
X = pd.concat([X.drop(['Embarked'], axis=1).reset_index(drop=True), embarked_df.reset_index(drop=True)], axis=1)


In [132]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X[['Age', 'Fare']] = scaler.fit_transform(X[['Age', 'Fare']])

In [133]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

#split into test train

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy Score", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy Score 0.8100558659217877
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.86      0.84       105
           1       0.79      0.74      0.76        74

    accuracy                           0.81       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.81      0.81      0.81       179

Confusion Matrix:
 [[90 15]
 [19 55]]


In [116]:

test_df = pd.read_csv('./datasets/test.csv')
test = test_df.drop(removed_col, axis=1)
test['Age'] = test['Age'].fillna(test['Age'].median())
test['Embarked'] = test['Embarked'].fillna(test['Embarked'].mode()[0]) 
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())
test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...
413,3,male,27.0,0,0,8.0500,S
414,1,female,39.0,0,0,108.9000,C
415,3,male,38.5,0,0,7.2500,S
416,3,male,27.0,0,0,8.0500,S


In [117]:
le = LabelEncoder()
test['Sex'] = le.fit_transform(test['Sex'])
ohe = OneHotEncoder(drop='first', sparse_output=False)
embarked_test_df = ohe.fit_transform(test[['Embarked']])
embarked_df = pd.DataFrame(embarked_test_df, columns=ohe.get_feature_names_out(['Embarked']))
final = pd.concat([test.drop(['Embarked'], axis=1).reset_index(drop=True), embarked_df.reset_index(drop=True)], axis=1)
final

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,3,1,34.5,0,0,7.8292,1.0,0.0
1,3,0,47.0,1,0,7.0000,0.0,1.0
2,2,1,62.0,0,0,9.6875,1.0,0.0
3,3,1,27.0,0,0,8.6625,0.0,1.0
4,3,0,22.0,1,1,12.2875,0.0,1.0
...,...,...,...,...,...,...,...,...
413,3,1,27.0,0,0,8.0500,0.0,1.0
414,1,0,39.0,0,0,108.9000,0.0,0.0
415,3,1,38.5,0,0,7.2500,0.0,1.0
416,3,1,27.0,0,0,8.0500,0.0,1.0


In [118]:
test_pred = model.predict(final)

In [119]:
submission = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived': test_pred})
submission.to_csv('submission.csv', index=False)

In [120]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/Users/arifbillah/.kaggle"
!kaggle competitions submit -c titanic -f submission.csv -m "Logistic Regression Submission"



100%|██████████████████████████████████████| 2.77k/2.77k [00:01<00:00, 2.58kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster

In [121]:
!kaggle competitions submissions -c titanic

fileName        date                 description                     status    publicScore  privateScore  
--------------  -------------------  ------------------------------  --------  -----------  ------------  
submission.csv  2025-03-25 22:05:17  Logistic Regression Submission  complete  0.76315                    
lr.csv          2025-03-25 22:02:59  Logistic Regression Submission  complete  0.76315                    
lr.csv          2025-03-25 21:38:34  Logistic Regression Submission  complete  0.76315                    
submission.csv  2025-03-25 21:34:34  Logistic Regression Submission  complete  0.76315                    
submission.csv  2025-03-25 21:27:41  Logistic Regression Submission  complete  0.29186                    
